In [ ]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI


In [ ]:
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"

load_dotenv(override=True)

google_api_key = os.getenv("GOOGLE_API_KEY")

if not google_api_key:
    print("No API key was found - please be sure to add your key to the .env file, and save the file! Or you can skip the next 2 cells if you don't want to use Gemini")
elif not google_api_key.startswith("AIz"):
    print("An API key was found, but it doesn't start AIz")
else:
    print("API key found and looks good so far!")


In [ ]:

# A class to represent a Webpage
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


marvel= Website("https://marvel.fandom.com/wiki/Marvel_Database")
print(marvel.title)
print(marvel.text)


In [ ]:
system_prompt = "You are a marvel assistant that analyzes the contents of the marvel website \
and provides a short summary about all the characters in Featured Articles in one or two lines, \
Respond in markdown."

# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

print(user_prompt_for(marvel))


In [ ]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

# To give you a preview -- calling OpenAI with system and user messages:
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)

messages_for(marvel)


In [ ]:
def summarize(url):
    website = Website(url)
    response = gemini.chat.completions.create(
        model = "gemini-2.5-flash-lite",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

summarize("https://marvel.fandom.com/wiki/Marvel_Database")


In [ ]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://marvel.fandom.com/wiki/Marvel_Database")